<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Lies die Zeilen der datei links.txt aus und gib die länge der liste aus
with open("links.txt", 'r') as f:
    links = [line.strip() for line in f.readlines()] #strip() entfernt \n 

print(links[0])

https://finance.yahoo.com/news/1-boj-focus-japans-progress-234938317.html


In [2]:
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import requests
import torch
import sqlite3

i = 0

for link in links:
    link = link.strip()
    res = requests.get(link)

    soup = BeautifulSoup(res.text, "html.parser")

    title = soup.find("title").text

    if(soup == None or title == 'Yahoo'):
        continue

    print(title)

    soup = soup.find("div", attrs={'class':'caas-body'})

    for div in soup.find_all('div'):
        div.unwrap()

    for ul in soup.find_all('ul'):
        # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
        p_tag = ul.find_previous_sibling('p')

        # Wenn ein <p>-Tag gefunden wurde, entferne es
        if p_tag:
            p_tag.decompose()

    # Entferne alle <div>-Elemente
    for ul in soup.find_all("ul"):
        ul.decompose()

    for button in soup.find_all("button"):
        button.decompose()

    for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
        p_tag.decompose()


    text = ""
    for p in soup.find_all('p'):
        text = text + p.text + "\n"

    # speichere das ergebnis in einer datenbank
    conn = sqlite3.connect('sentiment.db')
    c = conn.cursor()

    # Erstelle Tabelle mit Spalten id, title, link, text
    c.execute('''
    CREATE TABLE IF NOT EXISTS sentiment (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        link TEXT,
        text TEXT,
        )
    ''')

    # Bevor eine Zeile eingefügt wird, prüfen, ob sie bereits vorhanden ist, indem der Titel überprüft wird
    c.execute("SELECT * FROM sentiment WHERE title=?", (title,))

    if c.fetchone() != None:
        print("Already in database")
        continue

    # Füge eine neue Zeile mit den Werten title, link und text ein
    c.execute("INSERT INTO sentiment (title, link, text) VALUES (?, ?, ?, ?)", (title, link, text))
    
    conn.commit()
    conn.close()

    i = i + 1

print(f"Added {i} new articles to database")

UPDATE 1-BOJ to focus on Japan's progress in hitting price goal
Sentiment: neutral
Gina Rinehart’s Hancock, SQM Agree to Buy Azure for $1.1 Billion
Sentiment: bullish
Asia Shares to Track US Rally, Treasuries Ease: Markets Wrap
Sentiment: bullish
Clearlake, Insight Agree $4.4 Billion Deal to Buy Alteryx
Sentiment: bullish
Apple Plans Rescue for $17 Billion Watch Business in Face of Ban
Sentiment: neutral
For Gen X, a 'dream retirement' is a long shot
Sentiment: neutral
Wall Street Forced by Fed to Rethink 2024 Bond Yields
Sentiment: bearish
Adobe Has $6 Billion for AI and Buybacks After Figma Deal Collapses
Sentiment: bearish
Argentina Cuts Key Rate to 100% by Swapping Main Policy Tool
Sentiment: bullish
Record Southwest penalty not enough to protect passenger interests -consumer groups
Sentiment: bullish
Amazon in Talks With Diamond Sports for Streaming Deal, Investment
Sentiment: neutral
Can China get its economic miracle back on track in 2024?
Sentiment: bearish
Climate Change Is Ma

: 